# Computer Vision Nanodegree

## Project: Image Captioning

---

In this notebook, you will train your CNN-RNN model.  

You are welcome and encouraged to try out many different architectures and hyperparameters when searching for a good model.

This does have the potential to make the project quite messy!  Before submitting your project, make sure that you clean up:
- the code you write in this notebook.  The notebook should describe how to train a single CNN-RNN architecture, corresponding to your final choice of hyperparameters.  You should structure the notebook so that the reviewer can replicate your results by running the code in this notebook.  
- the output of the code cell in **Step 2**.  The output should show the output obtained when training the model from scratch.

This notebook **will be graded**.  

Feel free to use the links below to navigate the notebook:
- [Step 1](#step1): Training Setup
- [Step 2](#step2): Train your Model
- [Step 3](#step3): (Optional) Validate your Model

<a id='step1'></a>
## Step 1: Training Setup

In this step of the notebook, you will customize the training of your CNN-RNN model by specifying hyperparameters and setting other options that are important to the training procedure.  The values you set now will be used when training your model in **Step 2** below.

You should only amend blocks of code that are preceded by a `TODO` statement.  **Any code blocks that are not preceded by a `TODO` statement should not be modified**.

### Task #1

Begin by setting the following variables:
- `batch_size` - the batch size of each training batch.  It is the number of image-caption pairs used to amend the model weights in each training step. 
- `vocab_threshold` - the minimum word count threshold.  Note that a larger threshold will result in a smaller vocabulary, whereas a smaller threshold will include rarer words and result in a larger vocabulary.  
- `vocab_from_file` - a Boolean that decides whether to load the vocabulary from file. 
- `embed_size` - the dimensionality of the image and word embeddings.  
- `hidden_size` - the number of features in the hidden state of the RNN decoder.  
- `num_epochs` - the number of epochs to train the model.  We recommend that you set `num_epochs=3`, but feel free to increase or decrease this number as you wish.  [This paper](https://arxiv.org/pdf/1502.03044.pdf) trained a captioning model on a single state-of-the-art GPU for 3 days, but you'll soon see that you can get reasonable results in a matter of a few hours!  (_But of course, if you want your model to compete with current research, you will have to train for much longer._)
- `save_every` - determines how often to save the model weights.  We recommend that you set `save_every=1`, to save the model weights after each epoch.  This way, after the `i`th epoch, the encoder and decoder weights will be saved in the `models/` folder as `encoder-i.pkl` and `decoder-i.pkl`, respectively.
- `print_every` - determines how often to print the batch loss to the Jupyter notebook while training.  Note that you **will not** observe a monotonic decrease in the loss function while training - this is perfectly fine and completely expected!  You are encouraged to keep this at its default value of `100` to avoid clogging the notebook, but feel free to change it.
- `log_file` - the name of the text file containing - for every step - how the loss and perplexity evolved during training.

If you're not sure where to begin to set some of the values above, you can peruse [this paper](https://arxiv.org/pdf/1502.03044.pdf) and [this paper](https://arxiv.org/pdf/1411.4555.pdf) for useful guidance!  **To avoid spending too long on this notebook**, you are encouraged to consult these suggested research papers to obtain a strong initial guess for which hyperparameters are likely to work best.  Then, train a single model, and proceed to the next notebook (**3_Inference.ipynb**).  If you are unhappy with your performance, you can return to this notebook to tweak the hyperparameters (and/or the architecture in **model.py**) and re-train your model.

### Question 1

**Question:** Describe your CNN-RNN architecture in detail.  With this architecture in mind, how did you select the values of the variables in Task 1?  If you consulted a research paper detailing a successful implementation of an image captioning model, please provide the reference.

**Answer:** 


### (Optional) Task #2

Note that we have provided a recommended image transform `transform_train` for pre-processing the training images, but you are welcome (and encouraged!) to modify it as you wish.  When modifying this transform, keep in mind that:
- the images in the dataset have varying heights and widths, and 
- if using a pre-trained model, you must perform the corresponding appropriate normalization.

### Question 2

**Question:** How did you select the transform in `transform_train`?  If you left the transform at its provided value, why do you think that it is a good choice for your CNN architecture?

**Answer:** The default looks reasonable, although I have some doubts as to whether it makes sense to discard the data around the edge in this case. 

### Task #3

Next, you will specify a Python list containing the learnable parameters of the model.  For instance, if you decide to make all weights in the decoder trainable, but only want to train the weights in the embedding layer of the encoder, then you should set `params` to something like:
```
params = list(decoder.parameters()) + list(encoder.embed.parameters()) 
```

### Question 3

**Question:** How did you select the trainable parameters of your architecture?  Why do you think this is a good choice?

**Answer:** 

### Task #4

Finally, you will select an [optimizer](http://pytorch.org/docs/master/optim.html#torch.optim.Optimizer).

### Question 4

**Question:** How did you select the optimizer used to train your model?

**Answer:** 

In [6]:
import torch
import torch.nn as nn
from torchvision import transforms
import sys
sys.path.append('/opt/cocoapi/PythonAPI')
from pycocotools.coco import COCO
from data_loader import get_loader
from model import EncoderCNN, DecoderRNN
import math

import nltk
nltk.download('punkt')

## TODO #1: Select appropriate values for the Python variables below.
#batch_size=1
batch_size=10
#batch_size = 64          # batch size, we have 10GB of GPU memory, let's use it
vocab_threshold = 5        # minimum word count threshold
vocab_from_file = True    # if True, load existing vocab file
embed_size = 512           # dimensionality of image and word embeddings
hidden_size = 512          # number of features in hidden state of the RNN decoder
num_epochs = 25             # number of training epochs
save_every = 1             # determines frequency of saving model weights
print_every = 100          # determines window for printing average loss
log_file = 'training_log.txt'       # name of file with saved training loss and perplexity

# (Optional) TODO #2: Amend the image transform below.
transform_train = transforms.Compose([ 
    transforms.Resize(256),                          # smaller edge of image resized to 256
    transforms.RandomCrop(224),                      # get 224x224 crop from random location
    transforms.RandomHorizontalFlip(),               # horizontally flip image with probability=0.5
    transforms.ToTensor(),                           # convert the PIL Image to a tensor
    transforms.Normalize((0.485, 0.456, 0.406),      # normalize image for pre-trained model
                         (0.229, 0.224, 0.225))])

# Build data loader.
data_loader = get_loader(transform=transform_train,
                         mode='train',
                         batch_size=batch_size,
                         vocab_threshold=vocab_threshold,
                         vocab_from_file=vocab_from_file)

# The size of the vocabulary.
vocab_size = len(data_loader.dataset.vocab)

# Initialize the encoder and decoder. 
encoder = EncoderCNN(embed_size)
decoder = DecoderRNN(embed_size, hidden_size, vocab_size, max_batch_size=batch_size)

# Move models to GPU if CUDA is available. 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
encoder.to(device)
decoder.to(device)

# Define the loss function. 
criterion = nn.CrossEntropyLoss().cuda() if torch.cuda.is_available() else nn.CrossEntropyLoss()

# TODO #3: Specify the learnable parameters of the model.
params = list(decoder.parameters()) + list(encoder.embed.parameters()) + list(encoder.bn.parameters())

# TODO #4: Define the optimizer.
#optimizer = torch.optim.SGD(params, lr=0.01)

# this data is probably pretty sparse, and defaults are probably ok
#http://ruder.io/optimizing-gradient-descent/
#optimizer = torch.optim.Adam(params, lr=0.001)
optimizer = torch.optim.Adam(params)

# Set the total number of training steps per epoch.
total_step = math.ceil(len(data_loader.dataset.caption_lengths) / data_loader.batch_sampler.batch_size)

[nltk_data] Downloading package punkt to /home/sthenc/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Vocabulary successfully loaded from vocab.pkl file!
loading annotations into memory...


  0%|          | 801/414113 [00:00<00:51, 8009.63it/s]

Done (t=0.51s)
creating index...
index created!
Obtaining caption lengths...


100%|██████████| 414113/414113 [00:43<00:00, 9555.96it/s] 


<a id='step2'></a>
## Step 2: Train your Model

Once you have executed the code cell in **Step 1**, the training procedure below should run without issue.  

It is completely fine to leave the code cell below as-is without modifications to train your model.  However, if you would like to modify the code used to train the model below, you must ensure that your changes are easily parsed by your reviewer.  In other words, make sure to provide appropriate comments to describe how your code works!  

You may find it useful to load saved weights to resume training.  In that case, note the names of the files containing the encoder and decoder weights that you'd like to load (`encoder_file` and `decoder_file`).  Then you can load the weights by using the lines below:

```python
# Load pre-trained weights before resuming training.
encoder.load_state_dict(torch.load(os.path.join('./models', encoder_file)))
decoder.load_state_dict(torch.load(os.path.join('./models', decoder_file)))
```

While trying out parameters, make sure to take extensive notes and record the settings that you used in your various training runs.  In particular, you don't want to encounter a situation where you've trained a model for several hours but can't remember what settings you used :).

### A Note on Tuning Hyperparameters

To figure out how well your model is doing, you can look at how the training loss and perplexity evolve during training - and for the purposes of this project, you are encouraged to amend the hyperparameters based on this information.  

However, this will not tell you if your model is overfitting to the training data, and, unfortunately, overfitting is a problem that is commonly encountered when training image captioning models.  

For this project, you need not worry about overfitting. **This project does not have strict requirements regarding the performance of your model**, and you just need to demonstrate that your model has learned **_something_** when you generate captions on the test data.  For now, we strongly encourage you to train your model for the suggested 3 epochs without worrying about performance; then, you should immediately transition to the next notebook in the sequence (**3_Inference.ipynb**) to see how your model performs on the test data.  If your model needs to be changed, you can come back to this notebook, amend hyperparameters (if necessary), and re-train the model.

That said, if you would like to go above and beyond in this project, you can read about some approaches to minimizing overfitting in section 4.3.1 of [this paper](http://ieeexplore.ieee.org/stamp/stamp.jsp?arnumber=7505636).  In the next (optional) step of this notebook, we provide some guidance for assessing the performance on the validation dataset.

In [ ]:
import torch.utils.data as data
import numpy as np
import os
import requests
import time

# temporary
%load_ext autoreload
%autoreload 2

from model import EncoderCNN, DecoderRNN

# Open the training log file.
f = open(log_file, 'w')

## running the training locally
#old_time = time.time()
#response = requests.request("GET", 
#                            "http://metadata.google.internal/computeMetadata/v1/instance/attributes/keep_alive_token", 
#                            headers={"Metadata-Flavor":"Google"})


for epoch in range(1, num_epochs+1):

    for i_step in range(1, total_step+1):
        
        ## running the training locally
        #if time.time() - old_time > 60:
        #    old_time = time.time()
        #    requests.request("POST", 
        #                     "https://nebula.udacity.com/api/v1/remote/keep-alive", 
        #                     headers={'Authorization': "STAR " + response.text})

        # Randomly sample a caption length, and sample indices with that length.
        indices = data_loader.dataset.get_train_indices()
        # Create and assign a batch sampler to retrieve a batch with the sampled indices.
        new_sampler = data.sampler.SubsetRandomSampler(indices=indices)
        data_loader.batch_sampler.sampler = new_sampler

        # Obtain the batch.
        images, captions = next(iter(data_loader))

        # Move batch of images and captions to GPU if CUDA is available.
        images = images.to(device)
        captions = captions.to(device)

        # Zero the gradients.
        decoder.zero_grad()
        encoder.zero_grad()

        # Pass the inputs through the CNN-RNN model.
        features = encoder(images)
        outputs = decoder(features, captions)

        # Calculate the batch loss.
        loss = criterion(outputs.contiguous().view(-1, vocab_size), captions.contiguous().view(-1))

        # Backward pass.
        loss.backward(retain_graph=True)

        # Update the parameters in the optimizer.
        optimizer.step()

        # Get training statistics.
        stats = 'Epoch [%d/%d], Step [%d/%d], Loss: %.4f, Perplexity: %5.4f' % (epoch, num_epochs, i_step, total_step, loss.item(), np.exp(loss.item()))

        # Print training statistics (on same line).
        print('\r' + stats, end="")
        sys.stdout.flush()

        # Print training statistics to file.
        f.write(stats + '\n')
        f.flush()

        # Print training statistics (on different line).
        if i_step % print_every == 0:
            print('\r' + stats)

    # Save the weights.
    if epoch % save_every == 0:
        torch.save(decoder.state_dict(), os.path.join('./models', 'decoder-%d.pkl' % epoch))
        torch.save(encoder.state_dict(), os.path.join('./models', 'encoder-%d.pkl' % epoch))

# Close the training log file.
f.close()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Epoch [1/25], Step [100/41412], Loss: 4.1422, Perplexity: 62.94231
Epoch [1/25], Step [200/41412], Loss: 4.4320, Perplexity: 84.09834
Epoch [1/25], Step [300/41412], Loss: 3.5618, Perplexity: 35.22595
Epoch [1/25], Step [400/41412], Loss: 3.8433, Perplexity: 46.67939
Epoch [1/25], Step [500/41412], Loss: 4.3513, Perplexity: 77.57604
Epoch [1/25], Step [600/41412], Loss: 3.8125, Perplexity: 45.26140
Epoch [1/25], Step [700/41412], Loss: 4.0556, Perplexity: 57.7216
Epoch [1/25], Step [800/41412], Loss: 3.5645, Perplexity: 35.32114
Epoch [1/25], Step [900/41412], Loss: 3.6654, Perplexity: 39.07250
Epoch [1/25], Step [1000/41412], Loss: 3.5339, Perplexity: 34.2576
Epoch [1/25], Step [1100/41412], Loss: 4.2510, Perplexity: 70.17715
Epoch [1/25], Step [1200/41412], Loss: 3.1357, Perplexity: 23.00589
Epoch [1/25], Step [1300/41412], Loss: 3.0393, Perplexity: 20.89128
Epoch [1/25], Step [1400/41412], Loss: 

Epoch [1/25], Step [12100/41412], Loss: 2.4790, Perplexity: 11.9288
Epoch [1/25], Step [12200/41412], Loss: 2.3846, Perplexity: 10.85437
Epoch [1/25], Step [12300/41412], Loss: 2.3990, Perplexity: 11.0126
Epoch [1/25], Step [12400/41412], Loss: 2.3093, Perplexity: 10.0670
Epoch [1/25], Step [12500/41412], Loss: 2.6168, Perplexity: 13.6914
Epoch [1/25], Step [12600/41412], Loss: 2.5944, Perplexity: 13.3884
Epoch [1/25], Step [12700/41412], Loss: 2.6973, Perplexity: 14.8394
Epoch [1/25], Step [12800/41412], Loss: 2.5158, Perplexity: 12.3768
Epoch [1/25], Step [12900/41412], Loss: 3.8816, Perplexity: 48.5004
Epoch [1/25], Step [13000/41412], Loss: 3.4277, Perplexity: 30.8053
Epoch [1/25], Step [13100/41412], Loss: 2.7026, Perplexity: 14.9179
Epoch [1/25], Step [13200/41412], Loss: 2.3970, Perplexity: 10.99016
Epoch [1/25], Step [13300/41412], Loss: 2.6679, Perplexity: 14.4102
Epoch [1/25], Step [13400/41412], Loss: 2.5070, Perplexity: 12.2681
Epoch [1/25], Step [13500/41412], Loss: 3.7003

Epoch [1/25], Step [24100/41412], Loss: 2.5130, Perplexity: 12.3416
Epoch [1/25], Step [24200/41412], Loss: 2.4657, Perplexity: 11.77215
Epoch [1/25], Step [24300/41412], Loss: 2.7268, Perplexity: 15.2833
Epoch [1/25], Step [24400/41412], Loss: 2.8717, Perplexity: 17.6673
Epoch [1/25], Step [24500/41412], Loss: 2.5214, Perplexity: 12.4458
Epoch [1/25], Step [24600/41412], Loss: 2.7297, Perplexity: 15.3282
Epoch [1/25], Step [24700/41412], Loss: 2.1357, Perplexity: 8.46278
Epoch [1/25], Step [24800/41412], Loss: 2.4040, Perplexity: 11.0679
Epoch [1/25], Step [24900/41412], Loss: 2.9559, Perplexity: 19.2193
Epoch [1/25], Step [25000/41412], Loss: 2.7671, Perplexity: 15.9130
Epoch [1/25], Step [25100/41412], Loss: 2.4866, Perplexity: 12.0200
Epoch [1/25], Step [25200/41412], Loss: 2.7681, Perplexity: 15.9285
Epoch [1/25], Step [25300/41412], Loss: 2.5384, Perplexity: 12.6597
Epoch [1/25], Step [25400/41412], Loss: 2.2283, Perplexity: 9.284347
Epoch [1/25], Step [25500/41412], Loss: 2.0421

Epoch [1/25], Step [36100/41412], Loss: 2.6301, Perplexity: 13.8749
Epoch [1/25], Step [36200/41412], Loss: 2.2044, Perplexity: 9.06452
Epoch [1/25], Step [36300/41412], Loss: 2.2119, Perplexity: 9.13306
Epoch [1/25], Step [36400/41412], Loss: 2.2575, Perplexity: 9.55937
Epoch [1/25], Step [36500/41412], Loss: 2.7144, Perplexity: 15.09546
Epoch [1/25], Step [36600/41412], Loss: 2.0421, Perplexity: 7.70726
Epoch [1/25], Step [36700/41412], Loss: 2.6440, Perplexity: 14.0687
Epoch [1/25], Step [36800/41412], Loss: 2.3597, Perplexity: 10.5874
Epoch [1/25], Step [36900/41412], Loss: 2.3868, Perplexity: 10.8785
Epoch [1/25], Step [37000/41412], Loss: 2.9265, Perplexity: 18.6616
Epoch [1/25], Step [37100/41412], Loss: 2.6153, Perplexity: 13.6715
Epoch [1/25], Step [37200/41412], Loss: 2.3157, Perplexity: 10.1317
Epoch [1/25], Step [37300/41412], Loss: 3.2043, Perplexity: 24.6392
Epoch [1/25], Step [37400/41412], Loss: 2.6260, Perplexity: 13.8180
Epoch [1/25], Step [37500/41412], Loss: 2.3844,

Epoch [2/25], Step [6800/41412], Loss: 2.2863, Perplexity: 9.83810
Epoch [2/25], Step [6900/41412], Loss: 2.4959, Perplexity: 12.1325
Epoch [2/25], Step [7000/41412], Loss: 2.3972, Perplexity: 10.9929
Epoch [2/25], Step [7100/41412], Loss: 2.4437, Perplexity: 11.5152
Epoch [2/25], Step [7200/41412], Loss: 2.6927, Perplexity: 14.7718
Epoch [2/25], Step [7300/41412], Loss: 2.4505, Perplexity: 11.5943
Epoch [2/25], Step [7400/41412], Loss: 2.9593, Perplexity: 19.2854
Epoch [2/25], Step [7500/41412], Loss: 2.0666, Perplexity: 7.89799
Epoch [2/25], Step [7600/41412], Loss: 2.5432, Perplexity: 12.7202
Epoch [2/25], Step [7700/41412], Loss: 2.8001, Perplexity: 16.4465
Epoch [2/25], Step [7800/41412], Loss: 2.7084, Perplexity: 15.0049
Epoch [2/25], Step [7900/41412], Loss: 1.9845, Perplexity: 7.27520
Epoch [2/25], Step [8000/41412], Loss: 2.4657, Perplexity: 11.7723
Epoch [2/25], Step [8100/41412], Loss: 2.8080, Perplexity: 16.5769
Epoch [2/25], Step [8200/41412], Loss: 2.1231, Perplexity: 8.3

...

Epoch [3/25], Step [25500/41412], Loss: 2.7888, Perplexity: 16.2614
Epoch [3/25], Step [25600/41412], Loss: 2.6169, Perplexity: 13.6933
Epoch [3/25], Step [25700/41412], Loss: 2.4470, Perplexity: 11.55415
Epoch [3/25], Step [25800/41412], Loss: 2.3571, Perplexity: 10.5603
Epoch [3/25], Step [25900/41412], Loss: 2.8950, Perplexity: 18.0834
Epoch [3/25], Step [26000/41412], Loss: 2.1373, Perplexity: 8.47683
Epoch [3/25], Step [26100/41412], Loss: 2.4620, Perplexity: 11.7278
Epoch [3/25], Step [26200/41412], Loss: 2.6306, Perplexity: 13.8819
Epoch [3/25], Step [26300/41412], Loss: 2.4473, Perplexity: 11.5568
Epoch [3/25], Step [26400/41412], Loss: 3.9626, Perplexity: 52.5960
Epoch [3/25], Step [26500/41412], Loss: 2.1934, Perplexity: 8.96587
Epoch [3/25], Step [26600/41412], Loss: 2.1700, Perplexity: 8.75821
Epoch [3/25], Step [26700/41412], Loss: 2.0701, Perplexity: 7.92609
Epoch [3/25], Step [26800/41412], Loss: 2.2500, Perplexity: 9.48782
Epoch [3/25], Step [26900/41412], Loss: 3.0217,

Epoch [3/25], Step [37500/41412], Loss: 2.3822, Perplexity: 10.8289
Epoch [3/25], Step [37600/41412], Loss: 2.5925, Perplexity: 13.3636
Epoch [3/25], Step [37700/41412], Loss: 2.0378, Perplexity: 7.67349
Epoch [3/25], Step [37800/41412], Loss: 2.2043, Perplexity: 9.06356
Epoch [3/25], Step [37900/41412], Loss: 2.9560, Perplexity: 19.2216
Epoch [3/25], Step [38000/41412], Loss: 2.2462, Perplexity: 9.45205
Epoch [3/25], Step [38100/41412], Loss: 2.6027, Perplexity: 13.4996
Epoch [3/25], Step [38200/41412], Loss: 2.5518, Perplexity: 12.8301
Epoch [3/25], Step [38300/41412], Loss: 2.0543, Perplexity: 7.80128
Epoch [3/25], Step [38400/41412], Loss: 2.7086, Perplexity: 15.0077
Epoch [3/25], Step [38500/41412], Loss: 2.4575, Perplexity: 11.6753
Epoch [3/25], Step [38600/41412], Loss: 2.4888, Perplexity: 12.0471
Epoch [3/25], Step [38700/41412], Loss: 2.8279, Perplexity: 16.9097
Epoch [3/25], Step [38800/41412], Loss: 2.4316, Perplexity: 11.3772
Epoch [3/25], Step [38900/41412], Loss: 2.3986, 

Epoch [4/25], Step [8200/41412], Loss: 2.0105, Perplexity: 7.46684
Epoch [4/25], Step [8300/41412], Loss: 2.3020, Perplexity: 9.99371
Epoch [4/25], Step [8400/41412], Loss: 2.3079, Perplexity: 10.0533
Epoch [4/25], Step [8500/41412], Loss: 2.3430, Perplexity: 10.4126
Epoch [4/25], Step [8600/41412], Loss: 2.4679, Perplexity: 11.7982
Epoch [4/25], Step [8700/41412], Loss: 2.3321, Perplexity: 10.2993
Epoch [4/25], Step [8800/41412], Loss: 2.9283, Perplexity: 18.6960
Epoch [4/25], Step [8900/41412], Loss: 2.4557, Perplexity: 11.6546
Epoch [4/25], Step [9000/41412], Loss: 1.5559, Perplexity: 4.73952
Epoch [4/25], Step [9100/41412], Loss: 2.4726, Perplexity: 11.8528
Epoch [4/25], Step [9200/41412], Loss: 3.3235, Perplexity: 27.7575
Epoch [4/25], Step [9300/41412], Loss: 2.8062, Perplexity: 16.5474
Epoch [4/25], Step [9400/41412], Loss: 2.3938, Perplexity: 10.9553
Epoch [4/25], Step [9500/41412], Loss: 2.5484, Perplexity: 12.7871
Epoch [4/25], Step [9600/41412], Loss: 2.4267, Perplexity: 11.

Epoch [4/25], Step [20200/41412], Loss: 2.7448, Perplexity: 15.5622
Epoch [4/25], Step [20300/41412], Loss: 2.2745, Perplexity: 9.72336
Epoch [4/25], Step [20400/41412], Loss: 2.5626, Perplexity: 12.9689
Epoch [4/25], Step [20500/41412], Loss: 2.3588, Perplexity: 10.5784
Epoch [4/25], Step [20600/41412], Loss: 2.6318, Perplexity: 13.8982
Epoch [4/25], Step [20700/41412], Loss: 2.7133, Perplexity: 15.0794
Epoch [4/25], Step [20800/41412], Loss: 2.4071, Perplexity: 11.1017
Epoch [4/25], Step [20900/41412], Loss: 2.2557, Perplexity: 9.54156
Epoch [4/25], Step [21000/41412], Loss: 2.6481, Perplexity: 14.1270
Epoch [4/25], Step [21100/41412], Loss: 3.0284, Perplexity: 20.6652
Epoch [4/25], Step [21200/41412], Loss: 2.1851, Perplexity: 8.89135
Epoch [4/25], Step [21300/41412], Loss: 2.3214, Perplexity: 10.1904
Epoch [4/25], Step [21400/41412], Loss: 2.2493, Perplexity: 9.48119
Epoch [4/25], Step [21500/41412], Loss: 2.5432, Perplexity: 12.7205
Epoch [4/25], Step [21600/41412], Loss: 2.9304, 

Epoch [4/25], Step [32200/41412], Loss: 2.6857, Perplexity: 14.6682
Epoch [4/25], Step [32300/41412], Loss: 2.1404, Perplexity: 8.50291
Epoch [4/25], Step [32400/41412], Loss: 3.3425, Perplexity: 28.2899
Epoch [4/25], Step [32500/41412], Loss: 2.1670, Perplexity: 8.73194
Epoch [4/25], Step [32600/41412], Loss: 2.6494, Perplexity: 14.1455
Epoch [4/25], Step [32700/41412], Loss: 2.6463, Perplexity: 14.1022
Epoch [4/25], Step [32800/41412], Loss: 2.1177, Perplexity: 8.31228
Epoch [4/25], Step [32900/41412], Loss: 2.7479, Perplexity: 15.6092
Epoch [4/25], Step [33000/41412], Loss: 2.5661, Perplexity: 13.0149
Epoch [4/25], Step [33100/41412], Loss: 2.7575, Perplexity: 15.7604
Epoch [4/25], Step [33200/41412], Loss: 2.7023, Perplexity: 14.9140
Epoch [4/25], Step [33300/41412], Loss: 2.5327, Perplexity: 12.5876
Epoch [4/25], Step [33400/41412], Loss: 2.2648, Perplexity: 9.628985
Epoch [4/25], Step [33500/41412], Loss: 2.1775, Perplexity: 8.82467
Epoch [4/25], Step [33600/41412], Loss: 2.6672,

<a id='step3'></a>
## Step 3: (Optional) Validate your Model

To assess potential overfitting, one approach is to assess performance on a validation set.  If you decide to do this **optional** task, you are required to first complete all of the steps in the next notebook in the sequence (**3_Inference.ipynb**); as part of that notebook, you will write and test code (specifically, the `sample` method in the `DecoderRNN` class) that uses your RNN decoder to generate captions.  That code will prove incredibly useful here. 

If you decide to validate your model, please do not edit the data loader in **data_loader.py**.  Instead, create a new file named **data_loader_val.py** containing the code for obtaining the data loader for the validation data.  You can access:
- the validation images at filepath `'/opt/cocoapi/images/train2014/'`, and
- the validation image caption annotation file at filepath `'/opt/cocoapi/annotations/captions_val2014.json'`.

The suggested approach to validating your model involves creating a json file such as [this one](https://github.com/cocodataset/cocoapi/blob/master/results/captions_val2014_fakecap_results.json) containing your model's predicted captions for the validation images.  Then, you can write your own script or use one that you [find online](https://github.com/tylin/coco-caption) to calculate the BLEU score of your model.  You can read more about the BLEU score, along with other evaluation metrics (such as TEOR and Cider) in section 4.1 of [this paper](https://arxiv.org/pdf/1411.4555.pdf).  For more information about how to use the annotation file, check out the [website](http://cocodataset.org/#download) for the COCO dataset.

In [ ]:
# (Optional) TODO: Validate your model.